In [2]:
import sys

sys.path.append("..")
import pickle as pkl
from dataclasses import asdict, is_dataclass

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio
from dacite import from_dict
from ipywidgets import interact

from calibration.benchmark.benchmark import get_camera_from_entry, calibrate, evaluate
from calibration.benchmark.benchmark_result import (
    BenchmarkResult,
    calc_error,
    results_into_df,
)
from calibration.feature_detector.visualization import show_boards, show_corners
from calibration.feature_refiner.refine import refine_features, refine_features_single
from calibration.feature_refiner.visualization import show_refined_corners
from calibration.projector.board import gen_checkerboard_grid
from calibration.projector.camera import Camera

# pio.renderers.default = "svg"
from calibration.projector.projector import Projector
from calibration.solver.optimization.solve import solve as solve_optimization
from calibration.solver.scaramuzza.solve import solve as solve_scaramuzza

# from calibration.solver.solve import solve

%load_ext autoreload
%autoreload 2

In [6]:
# with open("../babelcalib_results_overlay.pkl", "rb") as f:
with open("../babelcalib_results_overlay.pkl", "rb") as f:
    results = pkl.load(f)
    
results = [r for r in results if r is not None and r.features is not None]

In [ ]:
results_ = refine_features(results)

Refining features:   0%|          | 0/784 [00:00<?, ?it/s]

In [ ]:
mask = {i for i, r in enumerate(results_) if r is not None}

In [ ]:
res = [results[i] for i in sorted(list(mask))]
res_ = [results_[i] for i in sorted(list(mask))]

In [ ]:
_solvers = [("Optimization", solve_optimization)]

In [ ]:
ents_and_feats = [(r.input, r.features) for r in res_]
solvers_args = [(f, get_camera_from_entry(e)) if f else None for e, f in ents_and_feats]
projs = calibrate(_solvers, solvers_args)
assert len(projs) == len(ents_and_feats)
return evaluate(ents_and_feats, projs)